## Libraries

In [ ]:
import ee
import sys
import geemap
import geemap.colormaps as cm

ee.Authenticate()
ee.Initialize()


from functools import partial


ACOLITE_PATH = r'C:\Users\sergi\Documents\repos\gee_acolite\src'

sys.path.append(ACOLITE_PATH)

import ACOLITE

## L2 from GEE with sen2cor

In [ ]:
# def to_rrs(image):
#     return image.divide(10_000)

# roi = ee.Geometry.LineString([[-5.39978,36.125207],[-5.350342,36.165106],[-5.289917,36.23156],[-5.213013,36.34219],[-5.158081,36.386398],[-5.064697,36.421746],[-4.949341,36.439414],[-4.822998,36.465909],[-4.702148,36.474738],[-4.608765,36.505634],[-4.526367,36.567388],[-4.487915,36.620281],[-4.438477,36.668734],[-4.383545,36.699552],[-4.312134,36.703954],[-4.174805,36.717157],[-4.081421,36.730358],[-3.982544,36.730358],[-3.883667,36.730358],[-3.779297,36.730358],[-3.685913,36.721557],[-3.587036,36.721557],[-3.521118,36.708355],[-3.389282,36.708355],[-3.312378,36.717157],[-3.22998,36.725958],[-3.164063,36.725958],[-3.054199,36.739157],[-2.982788,36.739157],[-2.889404,36.734757],[-2.807007,36.721557],[-2.702637,36.677984],[-2.647705,36.713199],[-2.598267,36.792375],[-2.532349,36.823143],[-2.356567,36.84072],[-2.318115,36.818748],[-2.213745,36.757196],[-2.164307,36.722001],[-2.098389,36.774787],[-2.04895,36.823143],[-1.983032,36.889034],[-1.928101,36.941705],[-1.867676,37.02941],[-1.801758,37.143276],[-1.774292,37.226377],[-1.73584,37.291918],[-1.675415,37.348675],[-1.587524,37.396666],[-1.461182,37.448985],[-1.362305,37.536102],[-1.246948,37.588324],[-1.098633,37.588324],[-0.97229,37.588324],[-0.900879,37.588324],[-0.818481,37.601373],[-0.725098,37.623118],[-0.725098,37.662241],[-0.725098,37.727402],[-0.76355,37.814194]])
# start, end, tile = ('2025-02-01', '2025-02-04', None)
    
# sentinel2_l2r_rrs = ACOLITE.search_with_clouds(roi, start, end, tile = tile, collection = 'S2_SR_HARMONIZED').map(to_rrs)

# print(f'Collection size {sentinel2_l2r_rrs.size().getInfo()}')

# sentinel2_l2w_rrs = sentinel2_l2r_rrs.map(ACOLITE.mask_water_by_SWIR1)
# for product in [ACOLITE.add_tur_nechad2016_665]:
#         sentinel2_l2w_rrs = sentinel2_l2w_rrs.map(product)

## GEE + ACOLITE Integration

In [ ]:
roi = ee.Geometry.LineString([[-5.39978,36.125207],[-5.350342,36.165106],[-5.289917,36.23156],[-5.213013,36.34219],[-5.158081,36.386398],[-5.064697,36.421746],[-4.949341,36.439414],[-4.822998,36.465909],[-4.702148,36.474738],[-4.608765,36.505634],[-4.526367,36.567388],[-4.487915,36.620281],[-4.438477,36.668734],[-4.383545,36.699552],[-4.312134,36.703954],[-4.174805,36.717157],[-4.081421,36.730358],[-3.982544,36.730358],[-3.883667,36.730358],[-3.779297,36.730358],[-3.685913,36.721557],[-3.587036,36.721557],[-3.521118,36.708355],[-3.389282,36.708355],[-3.312378,36.717157],[-3.22998,36.725958],[-3.164063,36.725958],[-3.054199,36.739157],[-2.982788,36.739157],[-2.889404,36.734757],[-2.807007,36.721557],[-2.702637,36.677984],[-2.647705,36.713199],[-2.598267,36.792375],[-2.532349,36.823143],[-2.356567,36.84072],[-2.318115,36.818748],[-2.213745,36.757196],[-2.164307,36.722001],[-2.098389,36.774787],[-2.04895,36.823143],[-1.983032,36.889034],[-1.928101,36.941705],[-1.867676,37.02941],[-1.801758,37.143276],[-1.774292,37.226377],[-1.73584,37.291918],[-1.675415,37.348675],[-1.587524,37.396666],[-1.461182,37.448985],[-1.362305,37.536102],[-1.246948,37.588324],[-1.098633,37.588324],[-0.97229,37.588324],[-0.900879,37.588324],[-0.818481,37.601373],[-0.725098,37.623118],[-0.725098,37.662241],[-0.725098,37.727402],[-0.76355,37.814194]])
start, end, tile = ('2025-02-01', '2025-02-04', None)

# pre processing
tolerance = 10
mask_clouds_by_probability = partial(ACOLITE.mask_clouds_by_probability, threshold = tolerance)
sentinel2_l1 = ACOLITE.search_with_clouds(roi, start, end, tile = tile).map(mask_clouds_by_probability)
print(f'Collection size {sentinel2_l1.size().getInfo()}')

sentinel2_l1r_rrs = sentinel2_l1.map(ACOLITE.select_sentinel2_bands).map(ACOLITE.to_rrs) # Digital Number to rrs
sentinel2_list = sentinel2_l1r_rrs.toList(sentinel2_l1r_rrs.size()) # Collection to List for iteration
# pre processing

# during
images = [ ACOLITE.dask_spectrum_fitting(ee.Image(sentinel2_list.get(i)))[0] for i in range(sentinel2_l1r_rrs.size().getInfo()) ]
sentinel2_l2r_rrs = ee.ImageCollection.fromImages( images )
# during

# post processing
sentinel2_l2w_rrs = sentinel2_l2r_rrs.map(ACOLITE.mask_water_by_SWIR1)

for product in [ACOLITE.add_tur_nechad2016_665]:
    sentinel2_l2w_rrs = sentinel2_l2w_rrs.map(product)
# post processing

## Plots

In [ ]:
tur_plot = {
    'palette': cm.palettes.BrBG_r,
    'min': 0,
    'max': 20,
}

rgb_plot = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.3,
}

# l1_image = sentinel2_l1r_rrs.mosaic()
l2w_image = sentinel2_l2w_rrs.mosaic().select(f'TUR_Nechad2016_665')

Map = geemap.Map()
# Map.addLayer(l1_image, rgb_plot, f"L1")
# Map.addLayer(l2w_image, tur_plot, f"Turbidity")

water_images = sentinel2_l2w_rrs.toList(sentinel2_l2w_rrs.size())
for i in range(sentinel2_l2w_rrs.size().getInfo()):
    image = ee.Image(water_images.get(i)).select(f'TUR_Nechad2016_665')
    Map.addLayer(image, tur_plot, f"Turbidity {i}")

Map.center_object(roi)
Map.add_colorbar(tur_plot, label="Turbidity (NTU)")

Map